# Design Patterns

## Introduction to Design Patterns
### Programming 2 / Master Data Science for the Life Scinces
### (c) 2020-2021 Martijn Herber / Stichting Hanzehogeschool Groningen
### License: Creative Commons BY-NC-SA 4.0 International

# Software Design
### Software Design is useful;
* Re-useability
* Maintainability
* Reliability
### Drawbacks;
* Not all designs fit the problem at hand
* Can become rigid and dogmatic ("Architecture Astronauts")

# Software Design: Paradigms
## ...here we go again with "paradigms"
### It's not just about _solving_ the problem (algorithm), but _how you write the solution_
### There's many ways of organizing software; just in Python we have;
1. Scripts (Files)
2. Modules (and directories!)
3. Functions
4. Data Types
5. *Classes*


# Class-based Design
## Two crucial Concepts
### 1. Data Encapsulation
You keep your data bound with the methods you apply to it
### 2. Separation of Concerns
Objects do 1 thing and 1 thing only; interact with each other via _interfaces_

# Class-based Design: Interfaces
## Say what? Inyerfaces?
* Interface: a standard way of interacting with an object defined by the class for _other objects_
    * I.e. this is for external consumption
* Method in Python that this is implemented; _the Method_
    * Even if it is through Python syntax sugar!
    * I.e. "dunder" __methods__
* I'm going to assume you know the material on classes from Prog 0!

# Class-based Design: Interface drawbacks in Python
* Note that in Python nothing prevents you from accessing all data and methods in a class!
* "Private" members of the class are designated with "_" or "__" prepended
* Using only "official" interface elements (as documented in e.g. the docstring!) is _convention_
* Guiding rule; never access an object's data directly, always use _methods_
    * Possible exception: using Python syntax ie. "[]", slicing or iteration

# Back to Design Patterns
## "Common" wisdom, "Typical" Solutions
* Design Patterns are nothing more than standard ways of solving problems
* Defined in the seminal 1994 book by Gamma, Helm, Johnson, Vlissides ("Gang of Four", GoF)
* For any object oriented language
    * But beware: some patterns are made obsolete by language syntax!
* Usually they need to be adapted to your specific use case
    * But if that takes too much time maybe you're doing it wrong!

# Design Patterns
* Major principle is: Don't do Object Orientation, to _Object-Based Programming_
* From the GoF book:
> Favor object composition over inheritance
* I.e. when you have a problem that has several varying parameters, make that multiple objects
* **Do not use multiple inheritance **

# A Design Pattern is a _Recipe_
1. Intent: describe what you're trying to solve
2. Motivation: why does the pattern solve the problem well?
3. Structure: what classes does the pattern consist of, what interfaces?
4. Code example

# Design Patterns: example
* Let's take a look at the Python "logging" module
* How would you design that?
* How about:

In [1]:
class Logger(object):
    def __init__(self, file):
        self.file = file

    def log(self, message):
        self.file.write(message + '\n')
        self.file.flush()

In [2]:
class SocketLogger(Logger):
    def __init__(self, sock):
        self.sock = sock

    def log(self, message):
        self.sock.sendall((message + '\n').encode('ascii'))

class SyslogLogger(Logger):
    def __init__(self, priority):
        self.priority = priority

    def log(self, message):
        syslog.syslog(self.priority, message)

### Problem?
* What if I want to add functionality like "filtering" of messages?
    * Indeed this is exactly one of many extra's the `logging` module provides
* Here's what that could look like:

In [3]:
class FilteredLogger(Logger):
    def __init__(self, pattern, file):
        self.pattern = pattern
        super().__init__(file)

    def log(self, message):
        if self.pattern in message:
            super().log(message)

# It works.

f = FilteredLogger('Error', sys.stdout)
f.log('Ignored: this is not important')
f.log('Error: but you want to see this')

NameError: name 'sys' is not defined

# So what's the problem again?
* What if you wanted to create a `FilteredSocketLogger` ?
* Why not create all combinations? 
1. `Logger`
2. `SocketLogger`
3. `SyslogLogger`
4. `FilteredLogger`
5. `FilteredSocketLogger`
6. `FilteeredSyslogLogger`
* This is what the GoF called "Subclass Explosion"
* Solution: _object composition_ under the "Single Responsibility Principle" or "Separation of Concerns"

# Design Pattern Solution: Adapter Pattern
* Key insignht: we don't change the already created classes but use an **Adapter** object to wrap them
* We do this by offering the right _interfaces_ ie. methods
    * Thank $DEITY for Duck Typing!
* So in our adapter:
1. We keep the Original Logger
2. We keep the FilteredLogger
3. We adapt each destination to the behavior of a file an then pass this adapter to the `Logger`

In [4]:
import socket

class FileLikeSocket:
    def __init__(self, sock):
        self.sock = sock

    def write(self, message_and_newline):
        self.sock.sendall(message_and_newline.encode('ascii'))

    def flush(self):
        pass

class FileLikeSyslog:
    def __init__(self, priority):
        self.priority = priority

    def write(self, message_and_newline):
        message = message_and_newline.rstrip('\n')
        syslog.syslog(self.priority, message)

    def flush(self):
        pass

In [5]:
sock1, sock2 = socket.socketpair()

fs = FileLikeSocket(sock1)
logger = FilteredLogger('Error', fs)
logger.log('Warning: message number one')
logger.log('Error: message number two')

print('The socket received: %r' % sock2.recv(512))

The socket received: b'Error: message number two\n'


# Phew! Subclass explosion averted!
## Just use an adapter patterns to mix and match any source to the logger

# Design Patterns: A Zoo of Patterns!
* For many of these kinds of problems, a pattern exists.
* Broadly divided into 3 classes;
    1. Creational Patterns
    2. Structural Patterns
    3. Behavioral Patterns
* Great site: [Refactoring Guru](https://refactoring.guru/design-patterns)

# Assignment: Pick a Pattern
### Pick a pattern on the "Refactoring Guru" website and explain it in class tomorrow!
* You don't have to write any code, you can select "Python" as the code example
* Another great (but a little dated) site is [Bruce Eckels "Python 3 Patterns, Recipes & Idioms"](https://python-3-patterns-idioms-test.readthedocs.io/en/latest/index.html)